# Prefix Tuning

## I. Presentation

For prefix tuning, the prompt was cancatenated to the key and value of the attention blocs instead of the embedding bloc.

transformers > 4.34
peft > 0.5
accelerate > 0.22

## II. Example

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # or "0,1" for multiple GPUs
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
# prepare task, we use the same example as butfit

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

ckp_data = "yahma/alpaca-cleaned"
ckp = "bigscience/bloomz-1b1"

# load dataset
data = load_dataset(ckp_data, split="train[:1000]")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(ckp)

# process data
def process(sample):

    MAX_LEN = 256

    human = tokenizer("Human: " + "\n".join([sample["instruction"], sample["input"]]).strip() + "\n\nAssistant: ")
    ml = tokenizer(sample["output"] + tokenizer.eos_token)

    input_ids = human["input_ids"] + ml["input_ids"]
    attention_mask = human["attention_mask"] + ml["attention_mask"]
    labels = [-100] * len(human["input_ids"]) + ml["input_ids"]

    if len(input_ids) > MAX_LEN:

        input_ids = input_ids[:MAX_LEN]
        attention_mask = attention_mask[:MAX_LEN]
        labels = labels[:MAX_LEN]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# tokenize dataset
tokenized_data = data.map(process, remove_columns=data.column_names)

# load model
model = AutoModelForCausalLM.from_pretrained(ckp, low_cpu_mem_usage=True)

# send to device
if torch.cuda.is_available():
    model = model.to("cuda:0")

2024-06-25 15:46:18.422939: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-25 15:46:18.423006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-25 15:46:18.425233: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-25 15:46:18.436381: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 15:46:20.522352: W tensorflow/compiler/tf2

In [3]:
# compute model size

params = sum(param.numel() for param in model.parameters())
print("model size: ", params/1e9, "GB")
print("total required memory: ", round(params/1e9 * (4 + 4 + 12), 2), "GB")

model size:  1.065314304 GB
total required memory:  21.31 GB


## III. prefix training

We use prefix tuning, but we don't add project layer for now.

In [4]:
# first we disable prefix project, which is similar to the soft prompt tuning
# We can remember that the soft prompt tuning has very limited capacity to influence the result.
# But with some projection layers, its performance can be improved, but with more trainable parameters.

from peft import PrefixTuningConfig, get_peft_model, TaskType

config = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10, prefix_projection=False)
config

PrefixTuningConfig(peft_type=<PeftType.PREFIX_TUNING: 'PREFIX_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, encoder_hidden_size=None, prefix_projection=False)

In [16]:
peft_model = get_peft_model(model, config)
peft_model

PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(250880, 1536)
      (word_embeddings_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=1536, out_features=4608, bias=True)
            (dense): Linear(in_features=1536, out_features=1536, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=1536, out_features=6144, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=6144, out_features=1536, bias=True)
          )
        )
      

In [6]:
peft_model.print_trainable_parameters()

trainable params: 737,280 || all params: 1,066,051,584 || trainable%: 0.0692


In [17]:
# compared to prompt tuning, the training is much slower since the trainable parameters are greater
# but the loss decreases more too.

# define training arguments
args = TrainingArguments(
    output_dir="./checkpoint",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=50,
    num_train_epochs=3
)

# define trainer
trainer = Trainer(
    model=peft_model,
    args=args,
    train_dataset=tokenized_data,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

# train
trainer.train()

Step,Training Loss
50,2.660200
100,2.022100
150,1.955900
200,2.040800
250,1.845200
300,1.822700
350,1.790800


TrainOutput(global_step=375, training_loss=2.0014372965494793, metrics={'train_runtime': 343.5229, 'train_samples_per_second': 8.733, 'train_steps_per_second': 1.092, 'total_flos': 1670056200806400.0, 'train_loss': 2.0014372965494793, 'epoch': 3.0})

In [18]:
# even though the loss (previous cell) decrease, the inference gives no-meaningful results.
# This may du to the fact that there are not enough training steps. With more training loop, the result
# may improve but it is very slow.

def generate(_model, _tokenizer, instruction, input=None):

    prompt = "human: {}\n{}".format(instruction, input).strip() + "\n\nAssistant: "
    inputs = _tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(_model.device)

    generation_output = _model.generate(
        input_ids=input_ids,
        output_scores=True,
        max_new_tokens=256
    )
    for seq in generation_output:
        output = tokenizer.decode(seq, skip_special_tokens=True)
        print(output)

generate(peft_model, tokenizer, "List five steps for comparing two products.")

human: List five steps for comparing two products.
None

Assistant: 1. Compare the two products by their brand name. 2. Compare the two products by their packaging. 3. Compare the two products by their ingredients. 4. Compare the two products by their packaging. 5. Compare the two products by their ingredients.


## IV. Projection layer

In P-Tuning, we saw that by adding a projection layer to the soft prompt layer, the results improved. Here, we share the same idea by adding a projection layer.

In [9]:
config = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10, prefix_projection=True)
config

PrefixTuningConfig(peft_type=<PeftType.PREFIX_TUNING: 'PREFIX_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, encoder_hidden_size=None, prefix_projection=True)

In [10]:
peft_model = get_peft_model(model, config)

In [11]:
# compared to when there is no projection layer, there are much more trainable parameters.

peft_model.print_trainable_parameters()
peft_model

trainable params: 115,696,128 || all params: 1,181,010,432 || trainable%: 9.7964


PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(250880, 1536)
      (word_embeddings_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=1536, out_features=4608, bias=True)
            (dense): Linear(in_features=1536, out_features=1536, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=1536, out_features=6144, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=6144, out_features=1536, bias=True)
          )
        )
      

In [12]:
peft_model.print_trainable_parameters()

trainable params: 115,696,128 || all params: 1,181,010,432 || trainable%: 9.7964


In [13]:
# define training arguments
args = TrainingArguments(
    output_dir="./checkpoint",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=50,
    num_train_epochs=3
)

# define trainer
trainer = Trainer(
    model=peft_model,
    args=args,
    train_dataset=tokenized_data,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

# train
trainer.train()

Step,Training Loss
50,2.660200
100,2.022100
150,1.955900
200,2.040800
250,1.845200
300,1.822700
350,1.790800


TrainOutput(global_step=375, training_loss=2.0014372965494793, metrics={'train_runtime': 334.7345, 'train_samples_per_second': 8.962, 'train_steps_per_second': 1.12, 'total_flos': 1670056200806400.0, 'train_loss': 2.0014372965494793, 'epoch': 3.0})

In [14]:
# compared to when there is no project layers, the results make much more sens now.
# The loss decreases significantly

generate(peft_model, tokenizer, "List five steps for comparing two products.")

human: List five steps for comparing two products.
None

Assistant: 1. Compare the ingredients of the two products. 2. Compare the active ingredients. 3. Compare the dosages of the two products. 4. Compare the claims of the two products. 5. Compare their reviews.


## V. Conclusion

With the projection layer, the trainable parameters increase a lot, but the training loss decreased as well.